# Sleep Staging Models

---

Links to notebooks in this repository:

[Quickstart Tutorial](./quickstart_tutorial.ipynb) | [Introduction](../../../../../Downloads/00_introduction.ipynb) | [Services](./01_services.ipynb) | [Sleep Staging](02_sleep_staging.ipynb) | [Ensembling Sleep Staging](./03_ensembling_sleep_staging.ipynb) | [Sleep Dynamics](./04_sleep_dynamics.ipynb) | [Luna Toolbox Integration](./05_luna_integration.ipynb)

---

In this notebook, we will present how to work with our tutorial EDF files (from the NSRR), as well as how to process your own uploaded data. We will also show how to practically run multiple models in a batch of EDF files you have moved in our shared `input` volume. We’ll use the same helper functions introduced in the previous [Services](./01_services.ipynb) notebook, always following the simple workflowfrom loading data, to harmonizing signal channels, to running predictions and visualizing the results.


> Helper function for interacting with the SLEEPYLAND services. By wrapping the HTTP POST logic in one function, we can easily send data/parameters to various endpoints of the `manager-api`, simplifying the code in the rest of the notebook.

In [ ]:
import requests
import shutil
import os
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go

# Define the base URL for the manager-api
MANAGER_API_BASE_URL = "http://manager-api:8989"

def make_post_request(endpoint, data=None, params=None):
    """
    Helper function to make a POST request to the specified endpoint.

    Parameters:
        endpoint (str): The API endpoint to hit.
        data (dict, optional): The form data to send in the request.
        params (dict, optional): The URL parameters to send in the request.

    Returns:
        dict: The JSON response if the request is successful.
    """
    url = f"{MANAGER_API_BASE_URL}/{endpoint}"
    response = requests.post(url, data=data, params=params)
    if response.status_code == 200:
        print("Success:", response.json())
        return response.json()
    else:
        print(f"Failed with status code {response.status_code}")
        return None

## Sleep staging on NSRR `learn` data

---

### Data loading


Let's first relocate the tutorial `.edf` and corresponding `.xml` files from their original exposed path into the shared `../input/learn/` folder. This ensures that all necessary files for the upcoming analyses are consolidated in one place accessible to the pipeline.

In [ ]:
# Define source and destination directories
source_dir = "../lunapi-notebooks/tutorial/edfs/"
destination_dir = "../input/learn/"

# Ensure destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Get a list of all .edf files in the source directory
edf_files = [f for f in os.listdir(source_dir) if f.endswith(".edf")]

# Copy each .edf file
for file in edf_files:
    shutil.copy(os.path.join(source_dir, file), destination_dir)

xml_files = [f for f in os.listdir(source_dir) if f.endswith(".xml")]

# Copy each .edf file
for file in xml_files:
    shutil.copy(os.path.join(source_dir, file), destination_dir)

print(f"Copied {len(edf_files)} EDF files successfully!")
print(f"Copied {len(xml_files)} XML files successfully!")

### Sleep staging predictions


In the block below we first use the get_channels function to determine which signal channels (EEG AND/OR EOG) are available for our `learn` dataset. We then pass those channel selections, along with the dataset name and a list of chosen models (e.g., `yasa` and `usleep`), to the `auto_evaluate_data` function. This automatically harmonizes the data (aligning and preparing signals) and runs sleep-stage predictions for all files in the learn dataset - producing ready-to-use results in the specified output folder (i.e., always retrievable from the `output` volume).

In [ ]:
def get_channels(dataset):
    """
    Retrieve the available EEG, EOG, and EMG channels for the specified dataset.
    Sends a request to fetch channel information based on the dataset name.

    :param dataset: Name of the dataset
    :return: Dictionary containing available channels
    """
    params = {'dataset': dataset}
    return make_post_request("get_channels", params=params)


def auto_evaluate_data(folder_root_name, output_folder_name, eeg_channels, eog_channels, emg_channels, dataset, models):
    """
    Perform both harmonization and prediction using the specified models.

    :param folder_root_name: Root folder containing the input data
    :param output_folder_name: Folder where results will be saved
    :param eeg_channels: List of EEG channels to use
    :param eog_channels: List of EOG channels to use
    :param emg_channels: List of EMG channels to use
    :param dataset: Name of the dataset
    :param models: List of models to apply for evaluation
    :return: Response from the evaluation request
    """
    data = {
        'folder_root_name': folder_root_name,
        'folder_name': output_folder_name,
        'eeg_channels': eeg_channels,
        'eog_channels': eog_channels,
        'emg_channels': emg_channels,
        'dataset': dataset,
        'models': models
    }
    return make_post_request("auto_evaluate", data=data)


def plot_confusion_matrix(cm, labels, title):
    """
    Plot a confusion matrix using Plotly.

    :param cm: Confusion matrix as a 2D array
    :param labels: List of class labels
    :param title: Title of the confusion matrix plot
    """
    fig = ff.create_annotated_heatmap(
        z=np.array(cm),
        x=labels, y=labels,
        colorscale='Blues',
        showscale=True
    )
    fig.update_layout(title=title, xaxis_title='Predicted', yaxis_title='Actual')
    fig.show()


def extract_metrics(data):
    """
    Extract evaluation metrics from the model results.

    :param data: List of dictionaries containing model evaluation results
    :return: Pandas DataFrame containing extracted metrics
    """
    results = []
    for model in data:
        for model_name, files in model.items():
            for file_data in files:
                file_name = file_data['file']
                metrics = file_data['metrics']
                results.append({
                    'Model': model_name,
                    'File': file_name,
                    'Accuracy': metrics['accuracy'],
                    'F1 Score': metrics['f1_score'],
                    'Cohen Kappa': metrics['cohen_kappa'],
                    'Recall': metrics['recall'],
                    'Precision': metrics['precision'],
                    'Confusion Matrix': metrics['cm']
                })
    return pd.DataFrame(results)


def visualize_results(data):
    """
    Display evaluation results and plot confusion matrices.

    :param data: Model evaluation results containing accuracy, precision, recall, etc.
    """
    df_metrics = extract_metrics(data)
    print(df_metrics[['Model', 'File', 'Accuracy', 'F1 Score', 'Cohen Kappa', 'Recall', 'Precision']])

    # Iterate through the extracted metrics and plot confusion matrices for each model-file combination
    for _, row in df_metrics.iterrows():
        plot_confusion_matrix(row['Confusion Matrix'], labels=["Wake", "N1", "N2", "N3", "REM"], title=f"{row['Model']} - {row['File']}")


# Retrieve the available channels for the dataset
# Channels could include EEG and/or EOG derivations depending on the dataset.
dataset = 'learn'
response = get_channels(dataset)

# Extract EEG, EOG, and EMG channels from the response
eeg_channels = response["eeg_channels"]
eog_channels = response["eog_channels"]
emg_channels = ['']  # Empty for now, can be updated as needed

# Define the models to be used for evaluation
models = ['yasa,usleep']

# Perform automatic evaluation using the selected models
response = auto_evaluate_data(dataset, 'output_learn', eeg_channels, eog_channels, emg_channels, dataset, models)

# Visualize the results from the evaluation
visualize_results(response)


### Hypnograms and hypnodensity graphs

Exploit the `create_hypnogram_predict` function to generate simple hypnograms based on the predicted sleep stages from each model. After loading the prediction files from the `output_learn` directory in the `output` volume, we convert the model outputs to stage labels and plot them over time. This helps you quickly visualize how each model classifies sleep stages throughout the night—no ground truth required.

In [ ]:
def create_hypnogram_evaluate(folder_name, models_selected):
    """
    This function compares predicted sleep stages with ground-truth annotations,
    plotting both on the same timeline for easy visual evaluation.
    """
    for model in models_selected:
        # Paths to the predicted outputs and true labels
        majority_folder = os.path.join('..', 'output', folder_name, model, 'majority')
        true_folder = os.path.join('..', 'output', folder_name, model, 'TRUE_files')

        # Gather the .npy files for predictions and for the ground truth
        majority_files = sorted([file for file in os.listdir(majority_folder) if file.endswith('.npy')])
        true_files = sorted(os.listdir(true_folder))

        for i, (maj_file, true_file) in enumerate(zip(majority_files, true_files)):
            # Load model predictions (argmax selects the stage with highest probability)
            sleep_stages_majority = np.load(os.path.join(majority_folder, maj_file)).argmax(-1).astype(int)
            # Load true labels (already in numeric form)
            sleep_stages_true = np.load(os.path.join(true_folder, true_file)).astype(int).ravel()

            # Remove threshold limit on printed output (for debugging if needed)
            np.set_printoptions(threshold=np.inf)

            # Each epoch is 30 seconds, so create a corresponding time axis
            time = np.arange(len(sleep_stages_majority)) * 30

            # Map numeric labels to textual sleep stage names
            sleep_stage_labels = ['Wake', 'NREM1', 'NREM2', 'NREM3', 'REM']
            sleep_stages_labels_majority = [sleep_stage_labels[stage] for stage in sleep_stages_majority]
            sleep_stages_labels_true = [sleep_stage_labels[stage] for stage in sleep_stages_true]

            # Assign colors to each stage index for a visually clear plot
            colors = {
                0: '#58e306',  # Wake
                1: '#2cf7f0',  # NREM1
                2: '#1173ef',  # NREM2
                3: '#4b4d4d',  # NREM3
                4: '#ee0e0e'   # REM
            }

            # Combine both predicted and true labels in a single figure
            fig_combined = go.Figure()

            # Plot predicted labels over time
            fig_combined.add_trace(go.Scatter(
                x=time,
                y=sleep_stages_labels_majority,
                mode='lines+markers',
                line=dict(color='#bdc2c3', width=2, shape='hv'),
                marker=dict(size=5, color=[colors[stage] for stage in sleep_stages_majority]),
                name='Pred'
            ))

            # Plot true labels over time
            fig_combined.add_trace(go.Scatter(
                x=time,
                y=sleep_stages_labels_true,
                mode='lines+markers',
                line=dict(color='#1f77b4', width=2, shape='hv'),
                marker=dict(size=5, color=[colors[stage] for stage in sleep_stages_true]),
                name='True'
            ))

            # Configure axes and title for clarity
            fig_combined.update_layout(
                title=f"{maj_file.split('.')[0].split('_')[0]} (Pred vs True) - {model}",
                xaxis=dict(title='Time (seconds)'),
                yaxis=dict(
                    title='Sleep Stage',
                    categoryorder='array',
                    categoryarray=['NREM3', 'NREM2', 'NREM1', 'REM', 'Wake']
                ),
                yaxis_range=[-0.5, 4.5]
            )

            # Display the interactive chart
            fig_combined.show()

# Call the function to compare predictions with ground truth for both 'usleep' and 'yasa'
create_hypnogram_evaluate("output_learn", ["usleep", "yasa"])

In [ ]:
def create_hypnodensity_graph(folder_name, models_selected, is_logits=False):
    """
    Generate a hypnodensity-style graph showing cumulative probability distributions
    across all sleep stages for each epoch.
    
    Parameters:
        folder_name (str): Name of the folder containing the output data.
        models_selected (list): List of model names to visualize.
        is_logits (bool): Flag indicating whether model outputs are raw logits
                          (requiring a softmax transform) or already probabilities.
    """
    for model in models_selected:
        # Path where prediction data (.npy files) is saved
        majority_folder = f'/app/output/{folder_name}/{model}/majority'

        # Collect all .npy files for the model from the majority folder
        majority_files = sorted([file for file in os.listdir(majority_folder) if file.endswith('.npy')])

        for i, maj_file in enumerate(majority_files):
            # Load the prediction probabilities (or logits) for each epoch
            sleep_probabilities_majority = np.load(os.path.join(majority_folder, maj_file))

            # If the model outputs logits, apply softmax here (example usage not shown)
            # if is_logits:
            #     sleep_probabilities_majority = softmax(sleep_probabilities_majority, axis=1)

            # Compute cumulative probabilities over the stages to create "stacked" areas
            cumulative_probs = np.cumsum(sleep_probabilities_majority, axis=1)

            colors = ['#364B9A', '#83B8D7', '#EAECCC', '#F99858', '#A50026']

            # Define names for each stage
            stage_names = ['Wake', 'N1', 'N2', 'N3', 'REM']

            fig = go.Figure()

            for j in range(cumulative_probs.shape[1]):
                fig.add_trace(go.Scatter(
                    x=np.arange(0, len(cumulative_probs)),
                    y=cumulative_probs[:, j],
                    mode='lines',
                    name=stage_names[j],
                    line=dict(width=2, color=colors[j]),  # Set line color
                    fill='tonexty',
                    fillcolor=f'rgba{tuple(int(colors[j][i:i + 2], 16) for i in (1, 3, 5)) + (0.5,)}',
                    # Convert HEX to RGBA
                    hoverinfo='none'
                ))

            # Configure the layout with titles and axis labels
            fig.update_layout(
                title=f"{maj_file.split('.')[0].split('_')[0]} - {model}",
                xaxis_title='Time (seconds)',
                yaxis_title='Cumulative Probability',
                yaxis_range=[0, 1],
                showlegend=True
            )

            # Display the plot
            fig.show()

# Example call to create hypnodensity graphs for the specified models
create_hypnodensity_graph("output_learn", ["usleep", "yasa"], False)


## Sleep staging on your own `edf`

---

In the second part of the notebook, we’ll show how to run SLEEPYLAND’s staging pipeline on your own uploaded `edf` files in few steps - no annotations needed. By following a similar procedure to the tutorial dataset, you can automatically harmonize your recordings, select relevant channels-type (EEG AND/OR EOG), and generate predictions using your model(s) of choice.

> NOTE: The system takes in input the channel type the user specify, then it automatically infer and extract all the recognised, e.g., EEG type, channels, forwarding them to the pre-trained models. Thus, the predictions in output are the result of all the combination of EEG AND/OR EOG channels the system recognized from the `edf` file. We suggest to use the majority vote predictions the system give in output.

In [ ]:
# Let's first remove/clean all files and subdirectories inside the input volume
# Define the directory path where files and folders need to be removed
directory = "/app/input"

# Iterate through all items in the directory
for item in os.listdir(directory):
    item_path = os.path.join(directory, item)
    if os.path.isfile(item_path):
        os.remove(item_path)
    elif os.path.isdir(item_path):
        shutil.rmtree(item_path)

### Data loading

Below is an example of how to use the `predict_on_my_edf` function with a single EDF file, `learn-nsrr01.edf`. First, we move the file to the shared `input` volume, ensuring that a dataset folder, in that case named `learn` exists (create the folder if necessary). Users should follow the same approach: first, choose/create a preferred root folder name located in the shared input volume, then move all the EDF files they wish to analyze into that folder before running predictions.


In [ ]:
# Define source and destination directories
source_dir = "../lunapi-notebooks/tutorial/edfs/"
destination_dir = "../input/learn/"

# Ensure destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Get a list of all .edf files in the source directory
edf_files = [f for f in os.listdir(source_dir) if f.endswith(".edf")]

# Copy one .edf file
file_to_copy = edf_files[0]
shutil.copy(os.path.join(source_dir, file_to_copy), destination_dir)

print(f"Copied EDF file successfully!")

## Sleep staging predictions

> **NOTE** - The exposed endpoint `predict_one` takes as input just **one** EDF file at a time.
> Below, we show how to run the prediction on a single EDF file.


In [ ]:
# Function to send prediction request for an EDF file
def predict_on_my_edf(folder_root_name, output_folder_name, channels_type, models):
    """
    Sends a request to perform prediction on an EDF file.

    Parameters:
    folder_root_name (str): Root directory containing the EDF file.
    output_folder_name (str): Directory where the prediction results will be saved.
    channels_type (list): List of channel types (e.g., EEG, EOG).
    models (list): List of models to use for prediction.
    """
    data = {
        'folder_root_name': folder_root_name,
        'folder_name': output_folder_name,
        'channels': channels_type,
        'models': models
    }
    make_post_request("predict_one", data=data)

In [ ]:
# Define the models to use for prediction
models = ['usleep']

# Define the channel types to use for prediction
channels_type = ['EEG', 'EOG']

# Define the dataset name
dataset = 'learn'

# Use the predict_on_my_edf function to perform prediction on the specified EDF file
predict_on_my_edf(dataset, 'output_my_edf', channels_type, models)